In [1]:
import tensorflow as tf
from tensorflow import keras
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

### Text Generator

In [2]:
shakespeare_url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
filepath = keras.utils.get_file("shakespeare.txt", shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()

In [3]:
print(shakespeare_text[:148])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?



All chars we have in data

In [4]:
"".join(sorted(set(shakespeare_text.lower())))

"\n !$&',-.3:;?abcdefghijklmnopqrstuvwxyz"

We train tokenizer to add number value to certain char

In [5]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(shakespeare_text)

Example:

In [6]:
tokenizer.texts_to_sequences(["First"])

[[20, 6, 9, 8, 3]]

In [7]:
tokenizer.sequences_to_texts([[20, 6, 9, 8, 3]])

['f i r s t']

In [8]:
max_id = len(tokenizer.word_index) # number of unique chars
dataset_size = tokenizer.document_count # number of whole chars 
max_id, dataset_size

(39, 1115394)

In [9]:
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1

train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

window_length - length of sample (101) \
shift - generate shift about 1 sample; example: 0-101, 1-102, 2-103, etc \
drop_reminder = True - let us have every sample with length of 101, when it's false every next sample is smaller about 1 

In [10]:
n_steps = 100
window_length = n_steps + 1 
dataset = dataset.repeat().window(window_length, shift=1, drop_remainder=True) # we have to created many examples, since now we have just one long example

In [11]:
dataset = dataset.flat_map(lambda window: window.batch(window_length)) # thanks to this we have every sample as different example

In [12]:
np.random.seed(42)
tf.random.set_seed(42)

Shuffle

In [13]:
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

OneHotEncoding

In [14]:
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))

In [15]:
dataset = dataset.prefetch(1)

### Model training

Training is taking too long for now ~1.5h per epoch

In [17]:
# model = keras.models.Sequential([
#     keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id],
#                      dropout=0.2, recurrent_dropout=0.2,
#                      ),
#     keras.layers.GRU(128, return_sequences=True,
#                      dropout=0.2, recurrent_dropout=0.2
#                     ),
#     keras.layers.TimeDistributed(keras.layers.Dense(max_id,
#                                                     activation="softmax"))
# ])
# model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
# history = model.fit(dataset, steps_per_epoch=train_size // batch_size,
#                     epochs=10)

In [19]:
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

Function to predict next char with extra parameter "temperature"

In [18]:
def next_char(text, temperature=1):
    X_new = preprocess([text])
    y_proba = model.predict(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

In [20]:
def complete_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

### State recursive network